# Overview: MSA data reduction in Yue+25



## Step 1: reducing the MSA spectra with the standard msaexp run.

The first step is a standard msaexp reduction.

### Requirements prior to running this notebook:

- Run the standard jwst reduction with Detector1Pipeline, i.e., with *_rate.fits files. Then, put the rate files in this directory.
- Download the *_msa.fits files to this directory.

In [1]:
import sys

# Quiet!
import warnings
warnings.filterwarnings("ignore")

import os

os.environ["CRDS_PATH"] = "/Users/minghao/Research/Projects/JWST/dependencies/crds_cache/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from grizli import jwst_level1
from msaexp import pipeline, slit_combine
import mastquery.jwst
import mastquery.utils

import os
import glob
import yaml
import warnings
import time

import numpy as np
import matplotlib.pyplot as plt
plt.ioff()

import astropy.time

import grizli
from grizli import utils, jwst_utils
jwst_utils.set_quiet_logging()
utils.set_warnings()

import astropy.io.fits as pyfits
from jwst.datamodels import SlitModel

import msaexp
from msaexp import pipeline
try:
    from msaexp import pipeline_extended
    HAS_EXTENDED_PIPELINE = True
except:
    HAS_EXTENDED_PIPELINE = False

import jwst
import time


In [2]:
# these functions are from Jorryt.
# I will clean it up later to make it easier to use.

def new_filename(rate_file, c='b'):
    """
    """
    spl = rate_file.split('_')
    spl[2] = c + spl[2][1:]
    return '_'.join(spl)


def preprocess_nirspec_file(rate_file, root, context='jwst_1225.pmap', as_fixed=False, clean=True, undo_flat=True, by_source=False, **kwargs):
    """
    Run preprocessing calibrations for a single NIRSpec exposure
    """
    from grizli import jwst_level1

    os.environ['CRDS_CONTEXT'] = os.environ['CRDS_CTX'] = context
    jwst_utils.set_crds_context()

    # print(rate_file, root)

    outroot = root

    file_prefix = rate_file.split('_rate')[0]
    # key = root + '-' + file_prefix
    key = f'{root}-{file_prefix}'

    WORKPATH = os.path.join(HOME, key)

    if not os.path.exists(WORKPATH):
        os.makedirs(WORKPATH)

    os.chdir(WORKPATH)

    _ORIG_LOGFILE = utils.LOGFILE
    _NEW_LOGFILE = os.path.join(WORKPATH, file_prefix + '_rate.log.txt')
    utils.LOGFILE = _NEW_LOGFILE

    msg = f"""# {rate_file} {root}
  jwst version = {jwst.__version__}
grizli version = {grizli.__version__}
msaexp version = {msaexp.__version__}
    """
    utils.log_comment(utils.LOGFILE, msg, verbose=True)

    # Download to working directory
    mastquery.utils.download_from_mast([rate_file], overwrite=False)

    os.system(f'aws s3 cp s3://grizli-v2/reprocess_rate/{rate_file} .')

    if not os.path.exists(rate_file):
        msg = f"Failed to download {rate_file}"
        utils.log_comment(utils.LOGFILE, msg, verbose=True)
        return 3

    with pyfits.open(rate_file) as im:
        if 'MSAMETFL' in im[0].header:
            msametf = im[0].header['MSAMETFL']
            mastquery.utils.download_from_mast([msametf], overwrite=False)
                
            msa = msaexp.msa.MSAMetafile(msametf)
            msa.merge_source_ids()
            msa.write(prefix='', overwrite=True)
                
        else:
            msametf = None

    use_file = rate_file
    use_prefix = use_file.split('_rate')[0]

    files = [use_file]
    files.sort()

    utils.log_comment(utils.LOGFILE, 'Reset DQ=4 flags', verbose=True)

    for _file in files:
        with pyfits.open(_file, mode='update') as im:
            # print(f'_file unset DQ=4')
            im['DQ'].data -= im['DQ'].data & 4
            im.flush()

    # Split into groups of 3 exposures
    groups = pipeline.exposure_groups(files=files, split_groups=True)

    print('Files:')
    print('======')
    print(yaml.dump(dict(groups)))

    # Single exposure groups
    single_exposure_groups = {}

    for g in groups:
        for exp, k in zip(groups[g], 'abcdefghijklmnopqrstuvwxyz'[:len(groups[g])]):
            gr = g.replace('-f', f'{k}-f').replace('-clear', f'{k}-clear')
            single_exposure_groups[gr] = [exp]


    pipes = []

    from jwst.assign_wcs.util import NoDataOnDetectorError

    source_ids = None
    pad = 0

    positive = False
    sources = None

    # Should just be one group....
    for g in groups:
        for exp, k in zip(groups[g], 'abcdefghijklmnopqrstuvwxyz'[:len(groups[g])]):
            mode = g.replace('-f', f'{k}-f').replace('-clear', f'{k}-clear')
            xmode = f'{mode}-fixed' if as_fixed else mode

            if sources is not None:
                source_ids = sources[g] #[3:6]
                if len(source_ids) < 1:
                    source_ids = None
            else:
                source_ids = None

            if os.path.exists(f'{xmode}.start'):
                print(f'Already started: {mode}')
                continue

            if outroot in ['macs0417-v1']:
                source_ids = None
                positive = False

            source_ids = None
            positive = False

            if not os.path.exists(f'{xmode}.slits.yaml'):#
                with open(f'{xmode}.start','w') as fp:
                    fp.write(time.ctime())

                    try:
                        pipe = pipeline.NirspecPipeline(mode=xmode,
                                                    files=single_exposure_groups[mode],
                                                    source_ids=source_ids,
                                                    pad=pad,
                                                    positive_ids=positive # Ignore background slits
                                                   )

                        pipe.full_pipeline(run_extractions=False,
                                           initialize_bkg=False,
                                           load_saved=None,
                                           scale_rnoise=False,
                                           fix_rows=False,
                                           )

                    except NoDataOnDetectorError:
                        print('NoDataOnDetectorError - skip')
                        pipe = None
                        
                    # continue

                # pipes.append(pipe)
                del(pipe)

                os.remove(f'{xmode}.start')
                print(f'{xmode} - Done! {time.ctime()}')

            else:
                print(f'Already completed: {mode}')

            os.system(f'cat {mode}.log.txt >> {_NEW_LOGFILE}')

            # break

    utils.LOGFILE = _NEW_LOGFILE

    if use_prefix != file_prefix:
        _USE_LOGFILE = os.path.join(WORKPATH, use_prefix + '_rate.log.txt')
        os.system(f"cp {_NEW_LOGFILE} {_USE_LOGFILE}")

    utils.LOGFILE = _ORIG_LOGFILE

    if clean:
        print('Clean up')
        files = glob.glob('*')
        for file in files:
            print(f'rm {file}')
            os.remove(file)

        os.chdir(HOME)
        os.rmdir(WORKPATH)

    return 2



In [3]:
# preprocessing the spectrum


# this is the directory where you save the data. I use the current directory.
HOME = os.getcwd()

LIST=glob.glob(HOME+'/jw04713001001_03101_0000[2-4]*nrs*_rate.fits') ####
for q in LIST:
    status = preprocess_nirspec_file(q,root='masq-j0100-visit1', clean=False, extend_wavelengths=False)



ENV CRDS_CONTEXT = jwst_1225.pmap
# /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00003_nrs2_rate.fits masq-j0100-visit1
  jwst version = 1.17.1
grizli version = 1.12.10
msaexp version = 0.9.10
    


fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


# (2025-09-24 03:59:44.150)
msaexp.msa: write from jw04713001001_01_msa.fits to jw04713001001_01_msa.fits
Reset DQ=4 flags
   N  value     
====  ==========
   1  jw04713001001-01-01-f070lp-g140m-nrs2
Files:
jw04713001001-01-01-f070lp-g140m-nrs2:
- /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00003_nrs2_rate.fits

Already started: jw04713001001-01-01a-f070lp-g140m-nrs2
ENV CRDS_CONTEXT = jwst_1225.pmap
# /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00004_nrs2_rate.fits masq-j0100-visit1
  jwst version = 1.17.1
grizli version = 1.12.10
msaexp version = 0.9.10
    


fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


jw04713001001_01_msa.fits   1 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits   1 : merge source_ids for slitlet  47 [7230, 7130]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet  47 [7230, 7130]
# (2025-09-24 03:59:47.607)
msaexp.msa: write from jw04713001001_01_msa.fits to jw04713001001_01_msa.fits
Reset DQ=4 flags
   N  value     
====  ==========
   1  jw04713001001-01-01-f070lp-g140m-nrs2
Files:
jw04713001001-01-01-f070lp-g140m-nrs2:
- /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00004_nrs2_rate.fits

# (2025-09-24 03:59:47.814)
msaexp.NirspecPipeline: Initialize jw04713001001-01-01a-f070lp-g140m-nrs2
msaexp.NirspecPipeline: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00004_nrs2_rate.fits
msaexp.NirspecPipeline: mode=jw04713001001-01-01a-f070lp-g140m-nrs2 exp_type=NRS_MSASPEC

...

jw04713001001_01_msa.fits   1 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits   1 : merge source_ids for slitlet  47 [7230, 7130]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet  47 [7230, 7130]
# (2025-09-24 04:01:29.302)
msaexp.msa: write from jw04713001001_01_msa.fits to jw04713001001_01_msa.fits
Reset DQ=4 flags
   N  value     
====  ==========
   1  jw04713001001-01-01-f070lp-g140m-nrs1
Files:
jw04713001001-01-01-f070lp-g140m-nrs1:
- /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00003_nrs1_rate.fits

# (2025-09-24 04:01:29.476)
msaexp.NirspecPipeline: Initialize jw04713001001-01-01a-f070lp-g140m-nrs1
msaexp.NirspecPipeline: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00003_nrs1_rate.fits
msaexp.NirspecPipeline: mode=jw04713001001-01-01a-f070lp-g140m-nrs1 exp_type=NRS_MSASPEC

fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


jw04713001001_01_msa.fits   1 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits   1 : merge source_ids for slitlet  47 [7230, 7130]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet  47 [7230, 7130]
# (2025-09-24 04:03:55.079)
msaexp.msa: write from jw04713001001_01_msa.fits to jw04713001001_01_msa.fits
Reset DQ=4 flags
   N  value     
====  ==========
   1  jw04713001001-01-01-f070lp-g140m-nrs1
Files:
jw04713001001-01-01-f070lp-g140m-nrs1:
- /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00004_nrs1_rate.fits

# (2025-09-24 04:03:55.256)
msaexp.NirspecPipeline: Initialize jw04713001001-01-01a-f070lp-g140m-nrs1
msaexp.NirspecPipeline: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00004_nrs1_rate.fits
msaexp.NirspecPipeline: mode=jw04713001001-01-01a-f070lp-g140m-nrs1 exp_type=NRS_MSASPEC

fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


jw04713001001_01_msa.fits   1 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits   1 : merge source_ids for slitlet  47 [7230, 7130]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet   5 [17352, 17400]
jw04713001001_01_msa.fits  49 : merge source_ids for slitlet  47 [7230, 7130]
# (2025-09-24 04:06:26.638)
msaexp.msa: write from jw04713001001_01_msa.fits to jw04713001001_01_msa.fits
Reset DQ=4 flags
   N  value     
====  ==========
   1  jw04713001001-01-01-f070lp-g140m-nrs1
Files:
jw04713001001-01-01-f070lp-g140m-nrs1:
- /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00002_nrs1_rate.fits

# (2025-09-24 04:06:26.850)
msaexp.NirspecPipeline: Initialize jw04713001001-01-01a-f070lp-g140m-nrs1
msaexp.NirspecPipeline: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/jw04713001001_03101_00002_nrs1_rate.fits
msaexp.NirspecPipeline: mode=jw04713001001-01-01a-f070lp-g140m-nrs1 exp_type=NRS_MSASPEC

This ends Step 1: msaexp reduction.

After running this, Please go to Step 2: copy msaexp data for PypeIt reduction.